In [ ]:
!git clone https://github.com/EmanuelRiquelme/fast_whisperx
%cd fast_whisperx
!pip install -e .

In [ ]:
!pip -q install openai pydub

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.0/77.0 kB 1.9 MB/s eta 0:00:00


In [ ]:
#you will probably need to authenticate to access the repository
!git clone https://github.com/goodangel1012/goodangel/tree/main/wav2lip
%cd wav2lip
!mkdir data
!pip3 install -r requirements.txt

You need to download this [weights](https://iiitaphyd-my.sharepoint.com/personal/radrabha_m_research_iiit_ac_in/_layouts/15/onedrive.aspx?id=%2Fpersonal%2Fradrabha%5Fm%5Fresearch%5Fiiit%5Fac%5Fin%2FDocuments%2FWav2Lip%5FModels%2Fwav2lip%5Fgan%2Epth&parent=%2Fpersonal%2Fradrabha%5Fm%5Fresearch%5Fiiit%5Fac%5Fin%2FDocuments%2FWav2Lip%5FModels&ga=1) and move them to wav2lip/checkpoints

In [ ]:
import whisperx
import torch
import time
import requests
import openai
import json
from pydub import AudioSegment
from concurrent.futures import ProcessPoolExecutor
openai.api_key  = ''
eleven_labels_api_key = ''

In [ ]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
language = 'en'
try:
    w_model = whisperx.load_model("tiny.en", device, compute_type='float16',language = language)
except:
    w_model = whisperx.load_model("tiny.en", device, compute_type='int8',language = language)

def generate_transcript(audio):
    """
    the first time it runs takes a few seconds to run, then after that takes faction of seconds.
    it could be a good improvement point, when it comes to build the optimizing the code on the the backend of the webapp
    """
    audio = whisperx.load_audio(audio)
    try:
        return w_model.transcribe(audio, batch_size=512)["segments"][0]['text']
    except:
        return w_model.transcribe(audio, batch_size=64)["segments"][0]['text']

In [ ]:
def mp3_to_wav(name_file):
    audio = AudioSegment.from_mp3(f'{name_file}.mp3')
    answer = 'data/answer.wav'
    audio.export(answer, format="wav")

def generate_audio(text,voice_id,model_id ='eleven_monolingual_v1'):
    CHUNK_SIZE = 400
    url = f"https://api.elevenlabs.io/v1/text-to-speech/{voice_id}/stream"
    headers = {
        "Accept": "audio/mpeg",
        "Content-Type": "application/json",
        "xi-api-key": eleven_labels_api_key
        }
    data = {
        "text": text,
        "model_id": model_id,
        'optimize_streaming_latency': 4,
        "voice_settings": {
            "stability": 0,
            "similarity_boost": 0
            }
        }
    response = requests.post(url, json=data, headers=headers)
    with open('output_new.mp3', 'wb') as f:
        for chunk in response.iter_content(chunk_size=CHUNK_SIZE):
                if chunk:
                        f.write(chunk)
    with ProcessPoolExecutor() as executor:
        executor.submit(mp3_to_wav, 'output_new')

def get_voice_id(voice_name):
    url = "https://api.elevenlabs.io/v1/voices"
    headers = {
        "Accept": "application/json",
        "xi-api-key": eleven_labels_api_key
    }
    response = requests.get(url, headers=headers)
    return [voice['voice_id'] for voice in json.loads(response.text)['voices'] if voice['name'] == voice_name][0]

def transcript_2_audio_response(transcript,voice_name):
    voice_id = get_voice_id(voice_name)
    transcript = f'{transcript} in 20 words at max '
    messages = [{"role": "user", "content": transcript}]
    model = 'gpt-3.5-turbo'
    response = openai.ChatCompletion.create(
        model=model,
        messages=messages,
        temperature=0,
        )
    text = response['choices'][0]['message']['content']
    generate_audio(text = text,voice_id = voice_id)


def question2answer(audio,voice_name):
    """
    returns a generated voice answering the generated response of the audio question.
    audio: is the direction where the audio file containing the question is located.
    voice_name: the name of the AI generated voice.
    """
    transcript = generate_transcript(audio)
    transcript_2_audio_response(transcript,voice_name)

In [ ]:
from os import listdir, path
import numpy as np
import scipy, cv2, os, sys, argparse, audio
import json, subprocess, random, string
from tqdm import tqdm
from glob import glob
import torch, face_detection
from models import Wav2Lip
import platform
import time
import subprocess
import shlex
import pandas as pd

In [ ]:
def get_smoothened_boxes(boxes, T):
    for i in range(len(boxes)):
        if i + T > len(boxes):
            window = boxes[len(boxes) - T:]
        else:
            window = boxes[i : i + T]
        boxes[i] = np.mean(window, axis=0)
    return boxes

def face_detect(images,face_det_batch_size):
    detector = face_detection.FaceAlignment(face_detection.LandmarksType._2D,
    flip_input=False)
    batch_size = face_det_batch_size
    while 1:
        predictions = []
        for i in tqdm(range(0, len(images), batch_size)):
            predictions.extend(detector.get_detections_for_batch(np.array(images[i:i + batch_size])))
        break
    results = []
    pady1, pady2, padx1, padx2 = [0, -1, 0, -1]
    for rect, image in zip(predictions, images):
        y1 = max(0, rect[1] - pady1)
        y2 = min(image.shape[0], rect[3] + pady2)
        x1 = max(0, rect[0] - padx1)
        x2 = min(image.shape[1], rect[2] + padx2)

        results.append([x1, y1, x2, y2])

    boxes = np.array(results)
    boxes = get_smoothened_boxes(boxes, T=5)
    results = [[image[y1: y2, x1:x2], (y1, y2, x1, x2)] for image, (x1, y1, x2, y2) in zip(images, boxes)]

    del detector
    return results

def datagen(frames, mels,wav2lip_batch_size,face_det_batch_size):
    img_size = 96
    img_batch, mel_batch, frame_batch, coords_batch = [], [], [], []

    face_det_results = face_detect(frames,face_det_batch_size) # BGR2RGB for CNN face detection
    for i, m in enumerate(mels):
        idx = i%len(frames)
        frame_to_save = frames[idx].copy()
        face, coords = face_det_results[idx].copy()

        face = cv2.resize(face, (img_size, img_size))

        img_batch.append(face)
        mel_batch.append(m)
        frame_batch.append(frame_to_save)
        coords_batch.append(coords)

        if len(img_batch) >= wav2lip_batch_size:
            img_batch, mel_batch = np.asarray(img_batch), np.asarray(mel_batch)

            img_masked = img_batch.copy()
            img_masked[:, img_size//2:] = 0

            img_batch = np.concatenate((img_masked, img_batch), axis=3) / 255.
            mel_batch = np.reshape(mel_batch, [len(mel_batch), mel_batch.shape[1], mel_batch.shape[2], 1])

            yield img_batch, mel_batch, frame_batch, coords_batch
            img_batch, mel_batch, frame_batch, coords_batch = [], [], [], []

    if len(img_batch) > 0:
        img_batch, mel_batch = np.asarray(img_batch), np.asarray(mel_batch)

        img_masked = img_batch.copy()
        img_masked[:, img_size//2:] = 0

        img_batch = np.concatenate((img_masked, img_batch), axis=3) / 255.
        mel_batch = np.reshape(mel_batch, [len(mel_batch), mel_batch.shape[1], mel_batch.shape[2], 1])

        yield img_batch, mel_batch, frame_batch, coords_batch

mel_step_size = 16

def load_model(checkpoint_path = 'checkpoints/wav2lip_gan.pth'):
    model = Wav2Lip()
    checkpoint = torch.load(checkpoint_path)
    s = checkpoint["state_dict"]
    new_s = {}
    for k, v in s.items():
        new_s[k.replace('module.', '')] = v
    model.load_state_dict(new_s)
    model = model.to('cuda:0')
    return model.eval()

In [ ]:
model  = load_model()
torch.backends.cudnn.benchmark = True

In [ ]:
def run_inference(face,audio_source = 'data/answer.wav',model = model,wav2lip_batch_size=1,face_det_batch_size=1,resize_factor=1,stream = False):
    """
    generates the avatar if stream is set to truth the avatar gets streamed and if false gets saved on the folder results.
    face: is a video or a picture of the person that will be used for the avatar (if it's a video the face of the person must be shown at all times)
    audio_source: is the voice of the avatar, it must be a wav file.
    face_det_batch_size = batch size of the face detection algorithm it's recommended to be set at 1, no matter the hardware.
    wav2lip_batch_size = batch size of the wav2lip algorithm it's recommended to be set at 1, no matter the hardware.
    resize_factor = downsampling factor. 1 equals to keeping the original resolution, increasing the number reduces the quality of the avatar
    but improves the speed of the function
    """
    """
    the first time it runs requires to download some weights so be mindful of that for production
    """
    face_name = face.split('/')[1].split('.')[0]
    output_name = f'{face_name}_{720 // resize_factor}p'
    if not face.split('.')[-1] in ['jpg', 'png', 'jpeg']:
        print('video')
        video_stream = cv2.VideoCapture(face)
        fps = video_stream.get(cv2.CAP_PROP_FPS)
        full_frames = []
        while 1:
            still_reading, frame = video_stream.read()
            if not still_reading:
                video_stream.release()
                break
            if resize_factor > 1:
                frame = cv2.resize(frame, (frame.shape[1]//resize_factor, frame.shape[0]//resize_factor))

            y1, y2, x1, x2 = [0, -1, 0, -1]
            if x2 == -1: x2 = frame.shape[1]
            if y2 == -1: y2 = frame.shape[0]

            frame = frame[y1:y2, x1:x2]

            full_frames.append(frame)
    else:
        full_frames = [cv2.imread(face)]
        fps = 25

    wav = audio.load_wav(audio_source, 16000)
    mel = audio.melspectrogram(wav)


    mel_chunks = []
    mel_idx_multiplier = 80./fps
    i = 0
    while 1:
        start_idx = int(i * mel_idx_multiplier)
        if start_idx + mel_step_size > len(mel[0]):
            mel_chunks.append(mel[:, len(mel[0]) - mel_step_size:])
            break
        mel_chunks.append(mel[:, start_idx : start_idx + mel_step_size])
        i += 1

    print("Length of mel chunks: {}".format(len(mel_chunks)))

    full_frames = full_frames[:len(mel_chunks)]

    batch_size = wav2lip_batch_size
    gen = datagen(full_frames.copy(), mel_chunks,face_det_batch_size = face_det_batch_size,wav2lip_batch_size = wav2lip_batch_size)

    for i, (img_batch, mel_batch, frames, coords) in enumerate(tqdm(gen,
                                            total=int(np.ceil(float(len(mel_chunks))/batch_size)))):
        if i == 0:
            frame_h, frame_w = full_frames[0].shape[:-1]
            out = cv2.VideoWriter('temp/result.avi',
                                    cv2.VideoWriter_fourcc(*'DIVX'), fps, (frame_w, frame_h))

        img_batch = torch.cuda.FloatTensor(np.transpose(img_batch, (0, 3, 1, 2)))
        mel_batch = torch.cuda.FloatTensor(np.transpose(mel_batch, (0, 3, 1, 2)))

        with torch.no_grad():
            pred = model(mel_batch, img_batch)

        pred = pred.cpu().numpy().transpose(0, 2, 3, 1) * 255.

        for p, f, c in zip(pred, frames, coords):
            y1, y2, x1, x2 = c
            p = cv2.resize(p.astype(np.uint8), (x2 - x1, y2 - y1))

            f[y1:y2, x1:x2] = p
            out.write(f)

    out.release()
    audio_source_quoted = shlex.quote(audio_source)
    command = f'ffmpeg -y -i {audio_source_quoted} -i temp/result.avi -c:v h264_nvenc -preset fast -c:a aac -strict -2 -qp 18 -threads 4 results/{output_name}.mp4'
    if not stream:


      # Execute the FFmpeg command
      try:
          subprocess.run(command, shell=True, check=True)
          print("FFmpeg operation completed successfully.")
      except subprocess.CalledProcessError as e:
          print(f"Error: {e}")
    else:
      # FFmpeg command with multithreading, CUDA hardware acceleration, and qp option, streaming the output

      # Execute the FFmpeg command and capture the output
      try:
          process = subprocess.Popen(command, shell=True, stdout=subprocess.PIPE, stderr=subprocess.PIPE)

          # Read and process the output stream (stdout and stderr) if needed
          output, error = process.communicate()

          # Your processing logic for the output stream goes here

          print("FFmpeg operation completed successfully.")
      except subprocess.CalledProcessError as e:
          print(f"Error: {e}")

You need to create upload a audio containing a question and run the function question2answer specifying the direction of the audio file, and the name of the AI generated voice.

Then you need to upload the video/picture of the avatar that we want to create and paste it in the directory wav2lip/data then run the function run_inference (read the comments in the function for more information)

example of how to create the avatar

In [ ]:
#let data/avatar_video.mp4 be the location where the video is located
#let data/question.wav be the the location where the audio with the question is located
question2answer(audio = data/question.wav,voice_name = 'Rachel')
main(face = data/avatar_video.mp4)